# Requirements


In [11]:
### PREREQUISTIES
### (many used only in one notebook...)

import os
import pandas as pd
import numpy as np
import pickle
import math
import random
import sys # to use your own functions
import io
import csv
from urllib.request import urlopen
import requests

import matplotlib.pyplot as plt

from scipy import stats

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

import sddk

In [2]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [3]:
### authorize google sheets
# to access our gsheet with data overview & metadata, you need a Google Service Account json file
# (see google-auth python package documentation to start with)
# I have mine located in my personal space on sciencedata.dk, so I read it from there:

# (1) read the file and parse its content
try:
    file_data = conf[0].get("https://sciencedata.dk/files/ServiceAccountsKey.json").json()
except:
    print("cannot find file ServiceAccountsKey.json")
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)
# (5) establish connection with spreadsheets specified by their url
sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/1boQal2mG8j_FK7XpxYM7iwIwK7Q_8ErTuor2_jC333o/edit?usp=sharing")

In [4]:
sh.worksheets()

[<Worksheet 'Sheet1' id:0>,
 <Worksheet 'seshat_axial_1000BC-1CE' id:1499875099>,
 <Worksheet 'latium_data' id:2003450966>]

# Variables

In [5]:
mg_vars = pd.read_csv("https://raw.githubusercontent.com/pesavage/moralizing-gods/master/variables.csv")
mg_vars.head(5)

,Variable,Subsection,ShortName,Category,VarNumber,Section
0,Polity Population,Social Scale,PolPop,scale,1,Social Complexity variables
1,Polity territory,Social Scale,PolTerr,scale,2,Social Complexity variables
2,Population of the largest settlement,Social Scale,CapPop,scale,3,Social Complexity variables
3,Administrative levels,Hierarchical Complexity,AdmLev,levels,4,Social Complexity variables
4,Military levels,Hierarchical Complexity,MilLev,levels,5,Social Complexity variables


In [6]:
mg_vars["Variable"].tolist()[:10]

['Polity Population',
 'Polity territory',
 'Population of the largest settlement',
 'Administrative levels',
 'Military levels',
 'Religious levels',
 'Settlement hierarchy',
 'Professional military officers',
 'Professional soldiers',
 'Professional priesthood']

In [7]:
mg_vars_dict = dict(zip(mg_vars["ShortName"], mg_vars["Variable"]))
mg_vars_dict

{'PolPop': 'Polity Population',
 'PolTerr': 'Polity territory',
 'CapPop': 'Population of the largest settlement',
 'AdmLev': 'Administrative levels',
 'MilLev': 'Military levels',
 'ReligLev': 'Religious levels',
 'SettlHier': 'Settlement hierarchy',
 'ProfOfficer': 'Professional military officers',
 'ProfSoldier': 'Professional soldiers',
 'ProfPriest': 'Professional priesthood',
 'FullTBur': 'Full-time bureaucrats',
 'ExamSyst': 'Examination system',
 'MeritProm': 'Merit promotion',
 'GovtBldg': 'Specialized government buildings',
 'Court': 'Courts',
 'LegCode': 'Formal legal code',
 'Judge': 'Judges',
 'Lawyer': 'Professional Lawyers',
 'Irrigation': 'irrigation systems',
 'WaterSuppl': 'drinking water supply systems',
 'Market': 'markets',
 'FoodStor': 'food storage sites',
 'Road': 'Roads',
 'Bridge': 'Bridges',
 'Canal': 'Canals',
 'Port': 'Ports',
 'Mine': 'Mines or quarries',
 'Courier': 'Couriers',
 'PostStation': 'Postal stations',
 'PostService': 'General postal service',
 

In [8]:
mg_vars_dict_inverse = dict(zip(mg_vars["Variable"], mg_vars["ShortName"]))
mg_vars_dict_inverse

{'Polity Population': 'PolPop',
 'Polity territory': 'PolTerr',
 'Population of the largest settlement': 'CapPop',
 'Administrative levels': 'AdmLev',
 'Military levels': 'MilLev',
 'Religious levels': 'ReligLev',
 'Settlement hierarchy': 'SettlHier',
 'Professional military officers': 'ProfOfficer',
 'Professional soldiers': 'ProfSoldier',
 'Professional priesthood': 'ProfPriest',
 'Full-time bureaucrats': 'FullTBur',
 'Examination system': 'ExamSyst',
 'Merit promotion': 'MeritProm',
 'Specialized government buildings': 'GovtBldg',
 'Courts': 'Court',
 'Formal legal code': 'LegCode',
 'Judges': 'Judge',
 'Professional Lawyers': 'Lawyer',
 'irrigation systems': 'Irrigation',
 'drinking water supply systems': 'WaterSuppl',
 'markets': 'Market',
 'food storage sites': 'FoodStor',
 'Roads': 'Road',
 'Bridges': 'Bridge',
 'Canals': 'Canal',
 'Ports': 'Port',
 'Mines or quarries': 'Mine',
 'Couriers': 'Courier',
 'Postal stations': 'PostStation',
 'General postal service': 'PostService',
 

In [9]:
rel_variables = ["SEFair", "SERecip",  "SEIngroup", "HighGods"]
for variable_short in rel_variables:
  print(mg_vars_dict[variable_short])


supernatural enforcement of fairness
supernatural enforcement of human reciprocity
supernatural enforcement of in-group loyalty
High Gods (creator gods)


# Latest version of the complete dataset

In [29]:
url = "http://seshatdatabank.info/equinox2020/data/download.csv"
response = requests.get(url).text
file_object = io.StringIO(response)
seshat_complete = pd.read_csv(file_object)

In [30]:
seshat_complete.head()

,NGA,Polity,Section,Subsection,Variable,Value.From,Value.To,Date.From,Date.To,Fact.Type,Value.Note,Date.Note
0,Kachi Plain,AfDurrn,General variables,NaN,RA,Stephen Dean,NaN,NaN,NaN,complex,list,NaN
1,Kachi Plain,AfDurrn,General variables,NaN,RA,Alice Williams,NaN,NaN,NaN,complex,list,NaN
2,Kachi Plain,AfDurrn,General variables,NaN,Original name,Durrani Empire,NaN,NaN,NaN,simple,simple,NaN
3,Kachi Plain,AfDurrn,General variables,NaN,Alternative names,Sadozai Kingdom,NaN,NaN,NaN,complex,list,NaN
4,Kachi Plain,AfDurrn,General variables,NaN,Alternative names,Last Afghan Empire,NaN,NaN,NaN,complex,list,NaN


In [31]:
seshat_complete.shape

(47396, 12)

In [14]:
seshat_complete[seshat_complete["NGA"]=="Latium"]

,NGA,Polity,Section,Subsection,Variable,Value.From,Value.To,Date.From,Date.To,Fact.Type,Value.Note,Date.Note
23954,Latium,ItLatBA,General variables,NaN,RA,Enrico Cioni,NaN,NaN,NaN,simple,simple,NaN
23955,Latium,ItLatBA,General variables,NaN,Original name,Latium - Bronze Age,NaN,NaN,NaN,simple,simple,NaN
23956,Latium,ItLatBA,General variables,NaN,Alternative names,Apennine culture,NaN,NaN,NaN,complex,list,NaN
23957,Latium,ItLatBA,General variables,NaN,Alternative names,Proto-Villanovan culture,NaN,NaN,NaN,complex,list,NaN
23958,Latium,ItLatBA,General variables,NaN,Duration,1800-900 BCE,NaN,NaN,NaN,simple,simple,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
26261,Latium,ItStPet,Religion and Normative Ideology,Moralizing Supernatural Powers,Moralizing religion adopted by elites,present,NaN,NaN,NaN,simple,simple,NaN
26262,Latium,ItStPet,Religion and Normative Ideology,Moralizing Supernatural Powers,Moralizing religion adopted by commoners,present,NaN,NaN,NaN,simple,simple,NaN
26263,Latium,ItStPet,Religion and Normative Ideology,Moralizing Supernatural Powers,Moralizing enforcement in afterlife,present,NaN,NaN,NaN,simple,simple,NaN
26264,Latium,ItStPet,Religion and Normative Ideology,Moralizing Supernatural Powers,Moralizing enforcement in this life,present,NaN,NaN,NaN,simple,simple,NaN


In [27]:
seshat_complete["Section"].unique().tolist()

['General variables',
 'Social Complexity variables',
 'Warfare variables',
 'Institutional Variables',
 'Social Mobility',
 'Religion and Normative Ideology']

In [28]:
seshat_complete[(seshat_complete["Polity"]=="TrRomDm") & (seshat_complete["Section"]=="Social Complexity variables")]

,NGA,Polity,Section,Subsection,Variable,Value.From,Value.To,Date.From,Date.To,Fact.Type,Value.Note,Date.Note
42807,Konya Plain,TrRomDm,Social Complexity variables,NaN,RA,Edward A L Turner,NaN,NaN,NaN,simple,simple,NaN
42808,Konya Plain,TrRomDm,Social Complexity variables,Social Scale,Polity territory,4000000,4500000,NaN,NaN,simple,range,NaN
42809,Konya Plain,TrRomDm,Social Complexity variables,Social Scale,Polity Population,40000000,70000000,300CE,NaN,simple,range,NaN
42810,Konya Plain,TrRomDm,Social Complexity variables,Social Scale,Population of the largest settlement,800000,NaN,NaN,NaN,simple,simple,NaN
42811,Konya Plain,TrRomDm,Social Complexity variables,Hierarchical Complexity,Settlement hierarchy,7,NaN,NaN,NaN,simple,simple,NaN
42812,Konya Plain,TrRomDm,Social Complexity variables,Hierarchical Complexity,Administrative levels,8,9,NaN,NaN,simple,range,NaN
42813,Konya Plain,TrRomDm,Social Complexity variables,Hierarchical Complexity,Religious levels,5,NaN,NaN,NaN,simple,simple,NaN
42814,Konya Plain,TrRomDm,Social Complexity variables,Hierarchical Complexity,Military levels,6,8,NaN,NaN,simple,range,NaN
42815,Konya Plain,TrRomDm,Social Complexity variables,Professions,Professional military officers,present,NaN,NaN,NaN,simple,simple,NaN
42816,Konya Plain,TrRomDm,Social Complexity variables,Professions,Professional soldiers,present,NaN,NaN,NaN,simple,simple,NaN


In [17]:
seshat_complete[(seshat_complete["Polity"]=="TrRomDm") & (seshat_complete["Section"]=="Religion and Normative Ideology")]

,NGA,Polity,Section,Subsection,Variable,Value.From,Value.To,Date.From,Date.To,Fact.Type,Value.Note,Date.Note
42915,Konya Plain,TrRomDm,Religion and Normative Ideology,NaN,RA,Enrico Cioni,NaN,NaN,NaN,simple,simple,NaN
42916,Konya Plain,TrRomDm,Religion and Normative Ideology,Deification of Rulers,Rulers are legitimated by gods,absent,NaN,NaN,NaN,simple,disputed,NaN
42917,Konya Plain,TrRomDm,Religion and Normative Ideology,Deification of Rulers,Rulers are legitimated by gods,present,NaN,NaN,NaN,simple,disputed,NaN
42918,Konya Plain,TrRomDm,Religion and Normative Ideology,Deification of Rulers,Rulers are gods,absent,NaN,NaN,NaN,simple,simple,NaN
42919,Konya Plain,TrRomDm,Religion and Normative Ideology,Normative Ideological Aspects of Equity and Pr...,Ideological reinforcement of equality,present,NaN,NaN,NaN,simple,simple,NaN
42920,Konya Plain,TrRomDm,Religion and Normative Ideology,Normative Ideological Aspects of Equity and Pr...,Ideological thought equates rulers and commoners,present,NaN,NaN,NaN,simple,simple,NaN
42921,Konya Plain,TrRomDm,Religion and Normative Ideology,Normative Ideological Aspects of Equity and Pr...,Ideological thought equates elites and commoners,absent,NaN,NaN,NaN,simple,disputed,NaN
42922,Konya Plain,TrRomDm,Religion and Normative Ideology,Normative Ideological Aspects of Equity and Pr...,Ideological thought equates elites and commoners,present,NaN,NaN,NaN,simple,disputed,NaN
42923,Konya Plain,TrRomDm,Religion and Normative Ideology,Normative Ideological Aspects of Equity and Pr...,Ideology reinforces prosociality,present,NaN,NaN,NaN,simple,simple,NaN
42924,Konya Plain,TrRomDm,Religion and Normative Ideology,Normative Ideological Aspects of Equity and Pr...,production of public goods,inferred absent,NaN,NaN,NaN,simple,simple,NaN


In [0]:
variables = seshat_complete["Variable"].unique()
variables

array(['RA', 'Editor', 'Settlement hierarchy', 'Administrative levels',
       'Religious levels', 'Military levels', 'Occupational complexity',
       'Professional military officers', 'Professional soldiers',
       'Professional priesthood', 'Full-time bureaucrats',
       'Source of support', 'Examination system', 'Merit promotion',
       'Specialized government buildings', 'Formal legal code', 'Judges',
       'Courts', 'Professional Lawyers', 'Communal buildings',
       'Special purpose house', 'Utilitarian public buildings',
       'irrigation systems', 'drinking water supply systems', 'markets',
       'food storage sites', 'Symbolic buildings',
       'Entertainment buildings', 'Knowledge/information buildings',
       'Roads', 'Bridges', 'Canals', 'Ports', 'Special purpose sites',
       'Ceremonial site', 'Burial site', 'Trading emporia', 'Enclosures',
       'Mines or quarries', 'Other site', 'Length', 'Area', 'Volume',
       'Weight', 'Time', 'Geometrical', 'Other', 'Mn

In [32]:
NGAs = seshat_complete["NGA"].unique()
NGAs

array(['Kachi Plain', 'Sogdiana', 'Middle Yellow River Valley',
       'Southern China Hills', 'North Colombia', 'Lowland Andes',
       'Upper Egypt', 'Cuzco', 'Yemeni Coastal Plain', 'Chuuk Islands',
       'Paris Basin', 'Deccan', 'Ghanaian Coast', 'Crete',
       'Big Island Hawaii', 'Kapuasi Basin', 'Central Java', 'Galilee',
       'Middle Ganga', 'Garo Hills', 'Southern Mesopotamia', 'Susiana',
       'Iceland', 'Latium', 'Kansai', 'Cambodian Basin',
       'Niger Inland Delta', 'Orkhon Valley', 'Valley of Oaxaca',
       'Basin of Mexico', 'Oro PNG', 'Lena River Valley', 'Konya Plain',
       'Cahokia', 'Finger Lakes'], dtype=object)

In [33]:
len(NGAs)

35